In [15]:
import pyscf
from pyscf import mcscf
import scipy.sparse
import openfermion as of
import numpy as np
from openfermionpyscf._run_pyscf import run_pyscf
from force_utility import gradient_mo_operator

In [2]:
def get_lih(positions, basis):
    atom = [['Li', tuple(positions[0:3])], ['H', tuple(positions[3:6])]]
    moldata = of.MolecularData(geometry=atom, basis=basis, charge=0, multiplicity=1)
    moldata = run_pyscf(moldata)
    return moldata

def get_hf_fd(positions, d, basis, occupied=None, active=None):
    f = []
    for idx in range(len(positions)):
        cd = positions.copy()
        cd[idx] += d
        hfwd = of.transforms.jordan_wigner(get_lih(cd, basis).get_molecular_hamiltonian(occupied_indices=occupied,active_indices=active))
        cd[idx] -= 2*d
        hbwd = of.transforms.jordan_wigner(get_lih(cd, basis).get_molecular_hamiltonian(occupied_indices=occupied,active_indices=active))
        f.append((hfwd-hbwd)/(2*delta))
    return f

# The Hellman-Feynman force operator
The Hellman-Feynman can be found as the derivative of the Hamiltonian matrix elements with respect to the position. We will use the example of $LiH$ and obtain the operator through finite differences and analytical derivation. 

In [3]:
coords = [0, 0, 0, 0, 0, 1.4]
#coords = [0, 0, 0, 0, 1.4, 1.4]/np.sqrt(2)
delta = 1e-5
occ = None
act= None
mol = get_lih(coords,'sto-3g')
h = of.transforms.jordan_wigner(mol.get_molecular_hamiltonian(occupied_indices=occ,active_indices=act))
eig, eigv = scipy.sparse.linalg.eigsh(of.get_sparse_operator(h), k=1, which='SA') 
f = get_hf_fd(coords, delta, 'sto-3g', occupied=occ, active=act)
print("FCI energy is {}".format(eig[0]))

FCI energy is -7.878453652277157


What can you tell about the difference between the number of terms in the Hamiltonian and forces? How many Pauli-words do they have in common? What does that mean for the measurements?

In [4]:
print("Number of pauli-words in the hamiltonian: {}".format(len(h.terms)))
for force in f:
    print("Number of pauli-words in the force: {}".format(len(force.terms)))
    common = 0
    for key in force.terms.keys():
        if key in h.terms.keys():
            common += 1
    print("Number of pauli-words in common with hamiltonian {}".format(common))

Number of pauli-words in the hamiltonian: 631
Number of pauli-words in the force: 0
Number of pauli-words in common with hamiltonian 0
Number of pauli-words in the force: 0
Number of pauli-words in common with hamiltonian 0
Number of pauli-words in the force: 467
Number of pauli-words in common with hamiltonian 467
Number of pauli-words in the force: 0
Number of pauli-words in common with hamiltonian 0
Number of pauli-words in the force: 0
Number of pauli-words in common with hamiltonian 0
Number of pauli-words in the force: 467
Number of pauli-words in common with hamiltonian 467


We can now evaluate this operator with respect to the FCI state. What do you expect for the values of the forces? Will they be equal of different for the different atoms? What with the acceleration?

In [5]:
h_eval= of.expectation(of.get_sparse_operator(h), eigv)
f_eval=[]
for force in f:
    if len(force.terms)==0:
        f_eval.append(0.)
    else:
        f_eval.append(of.expectation(of.get_sparse_operator(force), eigv))
print("Hamiltonian expectation: {}, Hamiltonian eigenvalue: {}".format(h_eval,eig[0]))
print(f_eval)
f_eval = [val.real for val in f_eval]

Hamiltonian expectation: (-7.878453652277166+0j), Hamiltonian eigenvalue: -7.878453652277157
[0.0, 0.0, (0.06101320993655044-3.469446951953614e-18j), 0.0, 0.0, (-0.06101321009632643+0j)]


Rotate the geometry by 45 degrees and repeat the excercise. What do you see?

# Force operators
We will now evaluate the force operators through the analytical formulas both with and without Pulay terms.

In [6]:
oei, tei = mol.get_integrals()
mymol = pyscf.gto.Mole(atom=[['Li', tuple(coords[0:3])],
                           ['H', tuple(coords[3:6])]],
                     basis = 'sto-3g')
mymol.build()
mf = pyscf.scf.RHF(mymol)
mf.kernel()
f_op = gradient_mo_operator(mymol, mf.mo_coeff, oei, tei)
f_op = [of.transforms.jordan_wigner(force) for force in f_op]
f_op_hf = gradient_mo_operator(mymol, mf.mo_coeff, oei, tei, with_pulay=False)
f_op_hf = [of.transforms.jordan_wigner(force) for force in f_op_hf]


converged SCF energy = -7.8605386610207


We can repeat the same analysis as before to see which terms are shared with the Hamiltonian. What do you notice? What does this mean for the cost of evaluation?

In [7]:
print("Number of pauli-words in the hamiltonian: {}".format(len(h.terms)))
for force in f_op_hf:
    print("Number of pauli-words in the force: {}".format(len(force.terms)))
    common = 0
    large = 0
    for key in force.terms.keys():
        if key in h.terms.keys():
            common += 1
    print("Number of pauli-words in common with hamiltonian {}".format(common))

Number of pauli-words in the hamiltonian: 631
Number of pauli-words in the force: 897
Number of pauli-words in common with hamiltonian 1
Number of pauli-words in the force: 897
Number of pauli-words in common with hamiltonian 1
Number of pauli-words in the force: 621
Number of pauli-words in common with hamiltonian 621
Number of pauli-words in the force: 897
Number of pauli-words in common with hamiltonian 1
Number of pauli-words in the force: 897
Number of pauli-words in common with hamiltonian 1
Number of pauli-words in the force: 621
Number of pauli-words in common with hamiltonian 621


In [8]:
print("Number of pauli-words in the hamiltonian: {}".format(len(h.terms)))
for force in f_op:
    print("Number of pauli-words in the force: {}".format(len(force.terms)))
    common = 0
    for key in force.terms.keys():
        if key in h.terms.keys():
            common += 1
    print("Number of pauli-words in common with hamiltonian {}".format(common))

Number of pauli-words in the hamiltonian: 631
Number of pauli-words in the force: 897
Number of pauli-words in common with hamiltonian 1
Number of pauli-words in the force: 897
Number of pauli-words in common with hamiltonian 1
Number of pauli-words in the force: 621
Number of pauli-words in common with hamiltonian 621
Number of pauli-words in the force: 897
Number of pauli-words in common with hamiltonian 1
Number of pauli-words in the force: 897
Number of pauli-words in common with hamiltonian 1
Number of pauli-words in the force: 621
Number of pauli-words in common with hamiltonian 621


We can now evaluate the force operators with the FCI state.

In [9]:
f_op_hf_eval=[]
for force in f_op_hf:
    if len(force.terms)==0:
        f_op_hf_eval.append(0.)
    else:
        f_op_hf_eval.append(of.expectation(of.get_sparse_operator(force), eigv))
print(f_op_hf_eval)
f_op_hf_eval = [val.real if np.abs(val) > 1e-8 else 0 for val in f_op_hf_eval]

[(3.2052659771623217e-16-6.162975822039155e-32j), (3.2305027992492573e-16-2.2186712959340957e-31j), (-0.005969170542868473+8.673617379884035e-19j), (-3.20526597716232e-16+1.1709654061874394e-31j), (-3.2305027992492553e-16+2.9582283945787943e-31j), (0.005969170542868684-8.673617379884035e-19j)]


In [10]:
f_op_eval=[]
for force in f_op:
    if len(force.terms)==0:
        f_op_eval.append(0.)
    else:
        f_op_eval.append(of.expectation(of.get_sparse_operator(force), eigv))
print(f_op_eval)
f_op_eval = [val.real if np.abs(val) > 1e-8 else 0 for val in f_op_eval]

[(2.322452563472421e-16+3.7594152514438844e-31j), (2.3519474845449173e-16+1.8488927466117464e-31j), (0.03350041368950895-1.734723475976807e-18j), (-2.3224525634724164e-16-2.9582283945787943e-31j), (-2.351947484544909e-16+9.860761315262648e-32j), (-0.033500413689508804-3.469446951953614e-18j)]


How do the numerical and analytical result for Hellman-Feynman compare? Can you improve the agreement? 

Are the Pulay terms negligible?

In [11]:
print("Hellman-Feynman evaluated through finite differences {}".format(f_eval))
print("Hellman-Feynman evaluated analytically {}".format(f_op_hf_eval))
print("Full force evaluated analytically {}".format(f_op_eval))


Hellman-Feynman evaluated through finite differences [0.0, 0.0, 0.06101320993655044, 0.0, 0.0, -0.06101321009632643]
Hellman-Feynman evaluated analytically [0, 0, -0.005969170542868473, 0, 0, 0.005969170542868684]
Full force evaluated analytically [0, 0, 0.03350041368950895, 0, 0, -0.033500413689508804]


Most qchem packages have a method to perform analytical gradients without going over operators. For this system, a CASSCF(6,4) calculation is equivalent to FCI and we can obtain the analytical gradient in a different way.

In [25]:
fci = mcscf.CASSCF(mf, 6, 4).run()
fci.nuc_grad_method().kernel()

CASSCF energy = -7.87845365227714
CASCI E = -7.87845365227714  E(CI) = -9.01240481853429  S^2 = 0.0000000
